<a href="https://colab.research.google.com/github/fabiabocayuva/Data_Visualization/blob/main/Visualiza%C3%A7%C3%A3o_com_Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Visualização de Dados de Vendas da Olist usando o Plotly**

### **Usando a ferramenta pip para instalação dos pacotes no Google Colab**

In [ ]:
!pip install cufflinks # cufflinks conecta a biblioteca pandas ao plot.ly
!pip install plotly

### **Importando as bibliotecas para uso offline do Plotly**

In [30]:
import cufflinks as cf
cf.go_offline()

import pandas as pd

import plotly
import plotly.graph_objs as go
import plotly.offline as py 
plotly.offline.init_notebook_mode(connected=True)
%matplotlib inline

O uso do Plotly no Google Colab requer a definição da função abaixo para exibição dos gráficos. 

Tal função deve ser chamada de forma offline a cada geração de gráfico.

In [31]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

### **Acessando e conhecendo a base de dados de vendas da Olist**

Antes de acessar a base de dados é importante que você já a tenha salva no drive. É possível encontrá-la facilmente na internet.

In [32]:
df = pd.read_csv('/content/drive/MyDrive/DataSciencedoZero/Dataframes Curso/olist_classified_public_dataset.csv')
df.head()

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class
0,0,1,delivered,89.99,14.38,1,1,2017-08-30 11:41:01.000000,2017-08-30 11:55:08.970352,2017-09-21 00:00:00.000000,2017-09-08 20:35:27.276847,Belo Horizonte,MG,308,beleza_saude,59,492,3,5,NaN,tudo certo,2017-09-09 00:00:00.000000,2017-09-10 03:27:54+00:00,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido
1,1,2,delivered,69.00,15.23,1,1,2017-09-26 09:13:36.000000,2017-09-26 09:28:10.922048,2017-10-24 00:00:00.000000,2017-09-29 21:13:04.984841,Pocos de Caldas,MG,377,brinquedos,50,679,4,5,NaN,o produto chegou antes de prazo parabens pela ...,2017-09-30 00:00:00.000000,2017-10-03 05:34:20+00:00,3,0,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido
2,2,3,delivered,99.80,15.86,2,4,2018-01-15 15:50:42.000000,2018-01-17 07:29:56.419769,2018-02-05 00:00:00.000000,2018-01-23 17:51:31.134866,Sao Jose dos Campos,SP,122,ferramentas_jardim,59,341,2,1,NaN,foi adquirido 6 itens desta loja onde até o mo...,2018-01-24 00:00:00.000000,2018-02-02 17:42:43+00:00,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega
3,3,4,delivered,87.00,12.74,1,1,2018-02-04 11:16:42.000000,2018-02-06 05:31:50.990164,2018-03-13 00:00:00.000000,2018-02-20 19:38:06.633080,Ribeirao Preto,SP,140,informatica_acessorios,45,411,1,4,NaN,achei a entrega um pouco demorada,2018-02-21 00:00:00.000000,2018-02-22 02:09:12+00:00,0,3,0,0,0,0,0,0,0,atrasado,problemas_de_entrega
4,4,5,delivered,99.90,17.95,1,2,2017-12-07 11:58:42.000000,2017-12-08 02:36:49.587515,2018-01-03 00:00:00.000000,2017-12-19 22:33:18.952512,RIO DE JANEIRO,RJ,205,cama_mesa_banho,60,189,1,3,NaN,comprei 02 jogos de lencois de cetim e uma col...,2017-12-20 00:00:00.000000,2017-12-23 04:17:03+00:00,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega


Checando a presença de missing values: só para conhecimento.

Como será feita apenas uma análise visual dos dados, os missing values não causam grande problema.

In [33]:
df.isnull().sum()

Unnamed: 0                          0
id                                  0
order_status                        0
order_products_value                0
order_freight_value                 0
order_items_qty                     0
order_sellers_qty                   0
order_purchase_timestamp            0
order_aproved_at                    0
order_estimated_delivery_date       0
order_delivered_customer_date     117
customer_city                       0
customer_state                      0
customer_zip_code_prefix            0
product_category_name               0
product_name_lenght                 0
product_description_lenght          0
product_photos_qty                  0
review_score                        0
review_comment_title             3584
review_comment_message              0
review_creation_date                0
review_answer_timestamp             0
votes_before_estimate               0
votes_delayed                       0
votes_low_quality                   0
votes_return

In [34]:
df.info() # visualizar o tipo das variáveis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3584 entries, 0 to 3583
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     3584 non-null   int64  
 1   id                             3584 non-null   int64  
 2   order_status                   3584 non-null   object 
 3   order_products_value           3584 non-null   float64
 4   order_freight_value            3584 non-null   float64
 5   order_items_qty                3584 non-null   int64  
 6   order_sellers_qty              3584 non-null   int64  
 7   order_purchase_timestamp       3584 non-null   object 
 8   order_aproved_at               3584 non-null   object 
 9   order_estimated_delivery_date  3584 non-null   object 
 10  order_delivered_customer_date  3467 non-null   object 
 11  customer_city                  3584 non-null   object 
 12  customer_state                 3584 non-null   o

*Dataset*

Pedidos: status, valor dos produtos, frete, quantidade de produtos, quantidade de vendedores, data/hora da realização do pedido, data/hora da aprovação do pedido, data/hora estimada da entrega, data/hora real da entrega.

Cliente: cidade, estado, CEP.

Produtos: categoria, tamanho do nome e descrição, quantidade de fotos.

Reviews: nota, titulo, mensagem, data de criação.

Classes dos reviews: classes mais votadas, subclasses mais votadas.

### **Quais informações interessantes esse dataset pode nos oferecer?**

* qual é a distribuição do status dos pedidos 
* em quais meses houveram maiores vendas (se tivesse uma meta, qual deles teria superado a meta)
* qual é o tempo médio de entrega
* como se distribuiu as vendas nos mês de menor e no mês de maior venda
* residentes de quais estados fizeram as maiores compras (gráfico de pizza)
* nota e classe dos reviews mais votados


**Distribuição do status dos pedidos**

In [35]:
configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=True)
df.order_status.value_counts().iplot(kind='bar', xTitle='Status', yTitle='Nº de Pedidos', title='Distribuição do Status dos Pedidos')

A maioria dos pedidos constam como entregues. Dos 3584 pedidos, apenas 25 foram cancelados.

**Quais meses do ano houveram maiores vendas?**

Transformando os dados das colunas de data/hora para o formato de datetime:

In [36]:
df.order_purchase_timestamp = pd.to_datetime(df.order_purchase_timestamp)
df.order_aproved_at = pd.to_datetime(df.order_aproved_at)
df.order_delivered_customer_date = pd.to_datetime(df.order_delivered_customer_date)
df.order_estimated_delivery_date = pd.to_datetime(df.order_estimated_delivery_date)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3584 entries, 0 to 3583
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Unnamed: 0                     3584 non-null   int64         
 1   id                             3584 non-null   int64         
 2   order_status                   3584 non-null   object        
 3   order_products_value           3584 non-null   float64       
 4   order_freight_value            3584 non-null   float64       
 5   order_items_qty                3584 non-null   int64         
 6   order_sellers_qty              3584 non-null   int64         
 7   order_purchase_timestamp       3584 non-null   datetime64[ns]
 8   order_aproved_at               3584 non-null   datetime64[ns]
 9   order_estimated_delivery_date  3584 non-null   datetime64[ns]
 10  order_delivered_customer_date  3467 non-null   datetime64[ns]
 11  customer_city    

Obtendo as informações de data em formato de meses e as guardando na coluna 'order_purchase_month' :

In [37]:
df['order_purchase_month'] = df.order_purchase_timestamp.dt.to_period('M').astype(str)
df.order_purchase_month.head()

0    2017-08
1    2017-09
2    2018-01
3    2018-02
4    2017-12
Name: order_purchase_month, dtype: object

Agrupando a soma dos valores dos produtos vendidos no mês na variável 'vendas_por_mes' :

In [38]:
vendas_por_mes = df.groupby(by='order_purchase_month').order_products_value.sum()
vendas_por_mes.head(100) # index são os meses

order_purchase_month
2017-02        9.99
2017-03      902.86
2017-04    17654.19
2017-05    25556.23
2017-06    34172.86
2017-07    47284.74
2017-08    39898.85
2017-09    45462.20
2017-10    40910.40
2017-11    55373.92
2017-12    43806.57
2018-01    46755.56
2018-02    42560.54
2018-03    33435.48
2018-04     3754.33
Name: order_products_value, dtype: float64

Duas formas interessantes de se plotar essa distribuição:

* Gráfico de linhas: dando um sentido acumulado 
* Gráfico de barras: dando um sentido comparativo entre os meses

In [39]:
configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=True)

max_vendas = vendas_por_mes.values.max()
min_vendas = vendas_por_mes.values.min()
cores = []
for x,y in zip(vendas_por_mes.values, vendas_por_mes.index):
  if x == max_vendas:
    mes_max_vendas = y
    cores.append('green')
  if x == min_vendas:
    mes_min_vendas = y
    cores.append('red')
  else:
    cores.append('lightpink')

trace0 = go.Scatter(x = vendas_por_mes.index,
                    y = vendas_por_mes.values,
                    marker = {'color':'blue'},
                    name = 'Gráfico de Linhas')

trace1 = go.Bar(x = vendas_por_mes.index,
                y = vendas_por_mes.values,
                marker = {'color': cores,
                          'line':{'color':'#333',
                                  'width':0.5}},
                opacity = 0.7,
                name = 'Gráfico de Barras')

data = [trace0, trace1]

layout = go.Layout(title = {'text':'Vendas por Período', 'x':0.5, 'y':0.9},
                   xaxis = {'title':'Período'},
                   yaxis = {'title':'Valor em Vendas'},
                   annotations = [{'text':'Mês destaque de vendas!',
                                   'x':mes_max_vendas,
                                   'y':max_vendas},
                                  {'text':'Pior mês de vendas',
                                   'x':mes_min_vendas,
                                   'y':min_vendas}
                                  ]
                   )

fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(range = ['order_purchase_month.min', 'order_purchase_month.max'])
py.iplot(fig)

**Qual é o tempo médio de entrega dos pedidos?**

- Tempo de entrega x quantidade de produtos por pedido x valor de frete

Calculando o tempo de entrega:

In [40]:
df['tempo_entrega'] = df['order_delivered_customer_date'] - df['order_aproved_at']
# transformando a coluna tempo_entrega em dias apenas
df['tempo_entrega'] = df['tempo_entrega'].dt.days
df.tempo_entrega.head()

0     9.0
1     3.0
2     6.0
3    14.0
4    11.0
Name: tempo_entrega, dtype: float64

In [41]:
configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=True)

trace = go.Scatter(x = df['tempo_entrega'],
                   y = df['order_freight_value'],
                   mode = 'markers',
                   marker = {'color':df['order_items_qty']},
                   text = 'Quantidade de pedidos:' + df['order_items_qty'].apply(str),
                   hoverinfo = 'text')

data = [trace]

layout = go.Layout(title = {'text':'<b>Quantidade de itens</b> e <b>Valor do Frete</b> pelo <b>Tempo de Entrega</b>', 'x':0.5, 'y':0.9},
                   xaxis = {'title':'Tempo de entrega'},
                   yaxis = {'title':'Valor do frete'})

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

Observa-se que não há grande relação entre o tempo de entrega, a quantidade de pedidos e o valor de frete. O intuitivo seria pensar que o valor do frete e a quantidade de itens de um pedido influenciaria no tempo de entrega, porém esses dados nos mostram tais parâmetros são independentes.

- Tempo de entrega x mês de compra

Agrupando a média do tempo de entrega por mês de compra e guardando na variável tempo_entrega_por_mes.

In [42]:
tempo_entrega_por_mes = df.groupby(by='order_purchase_month').tempo_entrega.mean()
tempo_entrega_por_mes.head(10)

order_purchase_month
2017-02    15.000000
2017-03    48.428571
2017-04    18.516484
2017-05    10.406061
2017-06    11.806452
2017-07    11.673684
2017-08    10.970684
2017-09    11.405145
2017-10    10.862295
2017-11    15.712560
Name: tempo_entrega, dtype: float64

In [43]:
configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=True)

trace = go.Bar(x = tempo_entrega_por_mes.index,
                y = tempo_entrega_por_mes.values,
                marker = {'line':{'color':'#333',
                          'width':0.5}},
                opacity = 0.7)

data = [trace]

layout = go.Layout(title = {'text':'Média do <b>Tempo de Entrega</b> por <b>Mês de Aquisição</b>', 'x':0.5, 'y':0.9},
                   xaxis = {'title':'Mês de Compra'},
                   yaxis = {'title':'Média do Tempo de Entrega'})

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

Com exceção ao mês de março de 2017, o tempo de entrega médio varia entre 10 a 20 dias em todos os meses. Alguma situação extraordinária deve ter ocorrido no mês de março de 2017 para que ele fuja do padrão nessa proporção.

- Tempo de entrega x estado do comprador x valor do frete

In [44]:
tempo_entrega_por_state = df.groupby(by='customer_state').tempo_entrega.mean()
tempo_entrega_por_state.head(10)

customer_state
AC    14.000000
AL    25.521739
AM    25.800000
BA    21.358621
CE    19.018868
DF    13.125000
ES    16.855072
GO    14.584416
MA    26.025641
MG    12.318707
Name: tempo_entrega, dtype: float64

In [45]:
valor_frete_por_state = df.groupby(by='customer_state').order_freight_value.mean()
valor_frete_por_state.head(10)

customer_state
AC    40.773333
AL    50.363750
AM    31.940000
BA    27.098733
CE    37.740690
DF    20.309324
ES    22.892143
GO    22.816071
MA    38.406098
MG    22.436644
Name: order_freight_value, dtype: float64

In [46]:
configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=True)
from plotly.subplots import make_subplots

trace = go.Bar(x = tempo_entrega_por_state.index,
               y = tempo_entrega_por_state.values,
               marker = {'line':{'color':'#333',
                                  'width':0.5},
                          'color': 'lightgreen'},
               opacity = 0.7,
               yaxis = 'y',
               offsetgroup = 1,
               name = 'tempo de entrega')

trace0 = go.Bar(x = valor_frete_por_state.index,
                y = valor_frete_por_state.values,
                marker = {'line':{'color':'#333',
                                  'width':0.5},
                          'color': 'orange'}, 
                opacity = 0.7,
                yaxis = 'y2',
                offsetgroup = 2,
                name = 'Valor do frete')

data = [trace, trace0]

layout = go.Layout(title = {'text':'Média do <b>Tempo de Entrega</b> e <b>Valor do Frete</b> por <b>Estado do Comprador</b>', 'x':0.5, 'y':0.9},
                   xaxis = {'title':'Estado do Comprador'},
                   yaxis = {'title':'Média do Tempo de Entrega (dias)', 'side':'left'},
                   yaxis2 = {'title':'Média do valor do Frete (R$)', 'overlaying': 'y', 'side':'right'})

fig = go.Figure(data = data, layout = layout)
fig.update_layout(barmode='group')
py.iplot(fig)

**Residentes de quais estados fizeram as maiores compras?**

In [47]:
valor_produtos_por_state = df.groupby(by='customer_state').order_products_value.sum()
valor_produtos_por_state = valor_produtos_por_state[valor_produtos_por_state > 10000]
valor_produtos_por_state.head(10)

customer_state
BA     22787.03
CE     11620.46
GO     11012.91
MG     55807.96
PE     11479.77
PR     31812.28
RJ     76222.26
RS     25735.71
SC     15419.26
SP    155670.58
Name: order_products_value, dtype: float64

In [48]:
configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=True)

trace = go.Pie(labels = valor_produtos_por_state.index, 
               values = valor_produtos_por_state.values,
               direction='clockwise',
               marker = {'line':{'color':'#000000', 'width':1}
                         },
               hoverinfo='label+percent+value',
               pull = [0,0,0,0,0,0,0,0,0,0.05],
               )

data = [trace]
layout = go.Layout(title = 'Compras por Estado')
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)